<a href="https://colab.research.google.com/github/Pager07/Webtech/blob/master/SVDHomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up

In [0]:
import pandas as pd 
import numpy as np
from scipy.sparse.linalg import svds

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive/', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


#Loading Book and Rating Table



In [0]:
root_dir = '/content/gdrive/My Drive/My Notes/3rd year/SSA/SSA/Web Technology/Webtech:Homework/BookData'
ratings_df = pd.read_csv(root_dir + '/ratev3_table.csv', sep=',', error_bad_lines=False,encoding="latin-1")
books_df = pd.read_csv(root_dir + '/books_table.csv',sep=',', error_bad_lines=False,encoding="latin-1")


In [4]:
ratings_df

,userID,ISBN,bookRating
0,2299,0805057706,10
1,2403,0345342968,9
2,2413,0679450408,6
3,2419,0679767800,7
4,2507,0385424736,9
...,...,...,...
3878,2146,0316769487,10
3879,2188,0870114573,7
3880,2227,051513306X,7
3881,2253,0805073337,9


In [5]:
books_df.head()

,ISBN,book_title,genre
0,0395366283,Curious George Goes to the Aquarium,Children's Fiction|General
1,0070434247,China: Its History and Culture,Asia - Civilization|Asian Studies - East Asia ...
2,2253141453,De l'amour et autres dÃ?ÃÂ©mons,Fiction
3,0425171299,Charmed,Romance fiction
4,0440508568,What Einstein Didn't Know : Scientific Answers...,Science|Essays


#Setting up Rating Data
 - Formate rating matrix 
    - user x movies 
    - use df.piviot ffunction
    - Fill cells with na with 0
- De-mean the data
    - nomrmalize by each users mean
        - find mean vector: user_mean * 1 
        - Subtract it from coloum
    - Convert it form a dataframe to numpy array

In [0]:
r_df = ratings_df.pivot(index='userID',columns='ISBN',values='bookRating').fillna(0)

In [7]:
r_df.head()

ISBN,0002211890,0002245663,0006161529,0006372570,000648381X,0007108389,000712614X,0020291604,002037500X,002086910X,002588591X,0026301253,0029227909,0030200881,0030576687,0030604176,0030635543,0043720455,0060002069,0060005424,0060090421,006009527X,0060163747,0060174986,0060176539,0060188901,0060192704,006019295X,006019300X,006019636X,0060198338,0060198834,0060201851,0060203889,006020432X,0060277327,0060279176,0060288884,006050563X,0060508663,...,2718605359,287714142X,2877302709,2901076181,315000067X,3150080126,3257061153,325721605X,340414743X,3421051496,3462015117,346202261X,3463401347,3491711061,3550070772,355008353X,360893099X,377010899X,3821808586,3860994565,3895430579,840101512X,8408033700,842044202X,8420444782,8420633194,8420633631,8423322912,842700611X,8430031979,843207036X,8437604184,843970206X,843970366X,847615562X,8478808175,848437386X,8830407445,8836807356,9728288115
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
R = r_df.as_matrix()
users_mean_rating = np.mean(R, axis = 1)
R_demeaned = R - users_mean_rating.reshape(-1,1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


#Singular Value Decomposition SVD
- Follow the guide
- Y = U x sigma x Vt
    - Kinda like X x Theta'
    - simga is a diagonal matrix like,identity matrix but not really
        - Just scales things up

- Y = user X movie

In [0]:
U,sigma,Vt = svds(R_demeaned, k=50)
sigma = np.diag(sigma)
Y = np.dot(np.dot(U,sigma),Vt) + users_mean_rating.reshape(-1,1)

In [0]:
Y_df = pd.DataFrame(Y,columns = r_df.columns)

In [11]:
Y_df.head()

ISBN,0002211890,0002245663,0006161529,0006372570,000648381X,0007108389,000712614X,0020291604,002037500X,002086910X,002588591X,0026301253,0029227909,0030200881,0030576687,0030604176,0030635543,0043720455,0060002069,0060005424,0060090421,006009527X,0060163747,0060174986,0060176539,0060188901,0060192704,006019295X,006019300X,006019636X,0060198338,0060198834,0060201851,0060203889,006020432X,0060277327,0060279176,0060288884,006050563X,0060508663,...,2718605359,287714142X,2877302709,2901076181,315000067X,3150080126,3257061153,325721605X,340414743X,3421051496,3462015117,346202261X,3463401347,3491711061,3550070772,355008353X,360893099X,377010899X,3821808586,3860994565,3895430579,840101512X,8408033700,842044202X,8420444782,8420633194,8420633631,8423322912,842700611X,8430031979,843207036X,8437604184,843970206X,843970366X,847615562X,8478808175,848437386X,8830407445,8836807356,9728288115
0,-0.005572,-0.004746,-0.005572,-0.004161,-0.005020,-0.004161,-0.005850,-0.004161,0.021200,-0.005066,-0.004320,-0.005793,-0.004189,-0.004161,0.017375,-0.004161,-0.004150,0.166983,-0.034242,0.022259,-0.029236,-0.008915,-0.134661,-0.006768,-0.010216,-0.029236,0.042728,0.006535,-0.004746,-0.010333,1.356791,-0.005831,-0.011347,-0.004320,-0.006920,-0.006260,0.026518,-0.004514,-0.004323,0.110270,...,-0.005342,-0.004320,-0.006179,-0.005342,-0.005153,-0.005342,-0.005887,-0.006891,-0.004514,-0.006554,0.127506,-0.014918,-0.005020,-0.007381,-0.003828,-0.006448,-0.004746,-0.005342,-0.004514,-0.005020,-0.007381,-0.005342,0.159572,-0.004746,-0.004034,-0.004514,-0.004161,-0.004161,-0.004161,-0.003937,-0.005020,0.204600,-0.005086,-0.004320,-0.004514,-0.004161,-0.004453,-0.005020,-0.005020,-0.005020
1,0.004536,0.004515,0.004536,0.004501,0.004522,0.004501,0.004543,0.004501,0.004406,0.003981,0.004505,0.004541,0.001279,0.004501,0.004338,0.004501,0.004500,0.000909,0.001364,0.000712,0.002622,0.000612,0.002825,0.004564,0.004308,0.002622,-0.002708,0.004000,0.004515,0.004563,0.003297,0.004004,0.004662,0.004505,0.002784,0.004552,0.004711,0.004510,0.000493,0.004159,...,0.004530,0.004505,0.004550,0.004530,0.004526,0.004530,0.004543,0.004567,0.004510,0.004559,0.001778,0.004734,0.004522,0.004578,0.004492,0.004552,0.004515,0.004530,0.004510,0.004522,0.004578,0.004530,0.001234,0.004515,0.004497,0.004510,0.004501,0.004501,0.004501,0.004495,0.004522,0.000307,0.004524,0.004505,0.004510,0.004501,0.004508,0.004522,0.004522,0.004522
2,0.007283,0.007249,0.007283,0.007225,0.007261,0.007225,0.007294,0.007225,0.007067,0.006357,0.007232,0.007292,0.001785,0.007225,0.006956,0.007225,0.007224,0.001385,0.002220,0.001204,0.004105,0.000980,0.004491,0.007330,0.006899,0.004105,-0.004740,0.006392,0.007249,0.007329,0.005179,0.006378,0.007494,0.007232,0.004301,0.007311,0.007579,0.007240,0.000764,0.006653,...,0.007274,0.007232,0.007308,0.007274,0.007266,0.007274,0.007296,0.007336,0.007240,0.007322,0.002797,0.007614,0.007261,0.007354,0.007211,0.007310,0.007249,0.007274,0.007240,0.007261,0.007354,0.007274,0.001910,0.007249,0.007219,0.007240,0.007225,0.007225,0.007225,0.007215,0.007261,0.000402,0.007263,0.007232,0.007240,0.007225,0.007237,0.007261,0.007261,0.007261
3,0.004573,0.004551,0.004573,0.004535,0.004558,0.004535,0.004580,0.004535,0.004431,0.003962,0.004539,0.004579,0.000898,0.004535,0.004360,0.004535,0.004534,0.000810,0.001421,0.000814,0.002492,0.000616,0.002788,0.004604,0.004317,0.002492,-0.003291,0.003989,0.004551,0.004603,0.003156,0.003962,0.004712,0.004539,0.002557,0.004591,0.004770,0.004544,0.000461,0.004156,...,0.004567,0.004539,0.004589,0.004567,0.004562,0.004567,0.004581,0.004607,0.004544,0.004599,0.001711,0.004792,0.004558,0.004620,0.004525,0.004591,0.004551,0.004567,0.004544,0.004558,0.004620,0.004567,0.001144,0.004551,0.004531,0.004544,0.004535,0.004535,0.004535,0.004528,0.004558,0.000182,0.004560,0.004539,0.004544,0.004535,0.004543,0.004558,0.004558,0.004558
4,0.007335,0.007298,0.007335,0.007272,0.007311,0.007272,0.007347,0.007272,0.007101,0.006330,0.007279,0.007345,0.001244,0.007272,0.006987

#Movie Reccomendation
 - Build a function that take userID and return you the sorted_user_prediction
    - Find which row the user is in?
         - I have made the dataset such that userID == row_index


In [0]:
def get_sorted_user_prediction(userId):
  user_row_number = userId 
  sorted_user_pred = Y_df.iloc[user_row_number].sort_values(ascending = False)
  return sorted_user_pred


In [14]:
get_sorted_user_prediction(4)

ISBN
0805073337    0.009630
0671721135    0.008907
0679404864    0.008879
0689859368    0.008860
156619024X    0.008478
                ...   
055358183X   -0.006082
0515133620   -0.006477
0812579569   -0.006552
0553801945   -0.008420
0670272663   -0.012694
Name: 4, Length: 1303, dtype: float64

In [0]:
def get_user_rated_books(books_dfx,ratings_dfx,userID):
  user_data = ratings_dfx[ratings_dfx.userID ==userID]
  user_full= user_data.merge(books_dfx,how='left',left_on = 'ISBN',right_on = 'ISBN').sort_values(['bookRating'],ascending=False)
  return user_full


In [0]:
def get_recommendation(books_df, ratings_df , userID):
  user_rated_books = get_user_rated_books(books_df,ratings_df,userID)
  books_not_rated_by_user = books_df[~books_df['ISBN'].isin(user_rated_books['ISBN'])]
  
  sorted_books_prediction_score_for_user = get_sorted_user_prediction(userID);
  sorted_books_prediction_score_for_user= pd.DataFrame(sorted_books_prediction_score_for_user).reset_index()
  highest_scored_unseen_books=(books_not_rated_by_user.merge(sorted_books_prediction_score_for_user,how='left',
                                                            left_on = 'ISBN',right_on = 'ISBN'));

  highest_scored_unseen_books = highest_scored_unseen_books.rename(columns={userID:'Prediction'}).sort_values('Prediction',ascending = False).iloc[:10, :-1]

  return user_rated_books , highest_scored_unseen_books



In [0]:
user_rated_books, highest_scored_unseen_books = get_recommendation(books_df, ratings_df, 20)

In [119]:
user_rated_books.head()

,userID,ISBN,bookRating,book_title,genre
0,20,0446527165,5,Wish You Well,Traffic Accident Victims|Family Relationships|...


In [120]:
highest_scored_unseen_books.head()

,ISBN,book_title,genre
1965,044020254X,The Empty House,Fiction|General
1963,0525938362,What I Lived for,City and town life|City and town life--New Yor...
529,067102096X,Lethal Seduction,Contemporary Romance|Women's Fiction
1635,0345455622,Lady Killer/Secret Admirer (2 Books in One),Paranormal & Fantastic Romance|Historical Romance
1161,0394557948,"Corpse Had a Familiar Face: Covering Miami, Am...","Buchanan, Edna|Journalists|Journalists--Florid..."


#Testing Different functions

In [56]:
user_data = ratings_df[ratings_df.userID ==494];user_data

,userID,ISBN,bookRating
2650,494,037540273X,8
2651,494,037575315X,5
2652,494,0375757902,5
2653,494,0393049922,10
2654,494,0449206513,5
2655,494,0515110469,5
2656,494,0618154000,5
2657,494,0670272663,5
2658,494,0670874868,5
2659,494,0679600000,5


In [62]:
user_full= user_data.merge(books_df,how='left',left_on = 'ISBN',right_on = 'ISBN').sort_values(['bookRating'],ascending=False);user_full

,userID,ISBN,bookRating,book_title,genre
3,494,0393049922,10,The Annotated Wizard of Oz: A Centennial Edition,"Wizard|Gale, Dorothy|Oz (Imaginary place)|Oz (..."
0,494,037540273X,8,Beloved : Gift Edition,Fiction|General
10,494,0679600922,5,"Snopes : The Hamlet, The Town, The Mansion (Mo...",Snopes Family (fictitious Characters)|Yoknapat...
16,494,0940450089,5,Nathaniel Hawthorne : Collected Novels: Fansha...,"Historical fiction, American|Literature"
15,494,0838315291,5,Charlotte Bronte: A Psychological Study (Engli...,"Psychology|Novelists, English|Fiction|Authorsh..."
14,494,081420256X,5,French and Italian Notebooks (Centenary Editio...,Literary Criticism|American
13,494,0811480542,5,Viva Mexico!: The Story of Benito Juarez and C...,Juvenile Literature|Presidents|Biography|Zapot...
12,494,080840475X,5,Hawthorne's Tales (Masterworks of Literature),Literary Collections
11,494,067960166X,5,The Basic Writings of Sigmund Freud (Psychopat...,Psychoanalytical Psychology
9,494,0679600000,5,Basic Writings of Nietzsche (Modern Library),Philosophy


In [0]:
userIDs = ratings_df.userID.unique()
xx = []
for userID in userIDs:
  user_data = ratings_df[ratings_df.userID == userID]
  if user_data.shape[0] > 4:
    xx.append(userID)

In [79]:
books_not_rated_by_user = books_df[~books_df['ISBN'].isin(user_full['ISBN'])]
sorted_books_prediction_score_for_user = get_sorted_user_prediction(494);
sorted_books_prediction_score_for_user= pd.DataFrame(sorted_books_prediction_score_for_user).reset_index()

highest_scored_unseen_books=(books_not_rated_by_user.merge(sorted_books_prediction_score_for_user,how='left',
                                                           left_on = 'ISBN',right_on = 'ISBN'));

highest_scored_unseen_books.rename(columns={494:'Prediction'}).sort_values('Prediction',ascending = False).iloc[:10, :-1]


,ISBN,book_title,genre
1178,067973452X,Notes from Underground (Vintage Classics),World Literature|Fiction Subjects|Peoples & Cu...
1913,0394709306,The Complete Grimm's Fairy Tales (Pantheon Fai...,"Fantasy fiction, Chinese--Translations into En..."
335,030723116X,Early Bird on Sesame Street,Literary Collections
439,038070384X,Lullaby,Fiction|Mystery & Detective
1822,076111937X,Amazing Gracie,Pets|Dogs
1492,0440505941,The Weekend Novelist,Fiction|Technique
212,157954519X,Ain't Misbehavin: The Groundbreaking Program f...,Pets|Cats
1840,0688088686,Tell Me No Secrets,Fiction|General
440,002086910X,OUT OF THE SILENT PLANET,Fiction|Science Fiction
2188,0895945398,Kitty Libber : Cat Cartoons by Women,Cats|Cats--Caricatures and cartoons|Wit and hu...
